In [112]:
# этот скрипт делает запрос к openai api.

In [113]:
#%pip install numpy
#%pip install pandas
#%pip install python-dotenv

In [114]:
import base64
import requests
from dotenv import load_dotenv
import os
import json

# Загрузка переменных из файла .env
load_dotenv()

# Получение значения переменной OPENAI_API_KEY
api_key = os.getenv('OPENAI_API_KEY')

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [115]:
# Path to your image
image_path = "img/1T.png"

# Getting the base64 string
base64_image = encode_image(image_path)

In [116]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "system", 
      "content": "You are a professional quantitative analyst specializing in financial market analysis and price" 
              + " forecasting. You always spend a few sentences explaining background context, assumptions, and" 
              + " step-by-step thinking BEFORE you try to answer a question to ensure accuracy. You conduct a"
              + " thorough analysis of the overall market situation before providing forecasts."
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": "Could you please analyze the attached minute candlestick chart and provide forecasts for the next"
                + " 5 minutes. The main chart includes Bollinger Bands (period=20, color=black), VWAP (color=blue), ATR"
                + " (period=10, color=red). The volume chart also includes RSI (period=14, color=black)."
                + " Below the volume chart is a panel with MACD (fast=12, slow=26, signal=9). Please MAKE SURE TO"
                + " INCLUDE the predicted CLOSING PRICE RANGE after the next 5 minutes as well as the PROBABILITIES of"
                + " price increase and decrease."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          },
        }
      ],
    }
  ],
  "max_tokens": 1000
}

# делаем первый запрос к API, в котором просим провести анализ графика.
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

{'id': 'chatcmpl-9VSVxzTRA5dyf6g81sx1i2N3KRgXR',
 'object': 'chat.completion',
 'created': 1717284805,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '### Analysis of the Minute Candlestick Chart\n\n#### Background and Indicators\n1. **Bollinger Bands (Black, Period=20)**: These provide a measure of volatility, indicating potential overbought and oversold conditions.\n2. **VWAP (Blue)**: The Volume Weighted Average Price helps understand the true average price a stock has traded at throughout the day, considering volume.\n3. **ATR (Red, Period=10)**: The Average True Range measures market volatility.\n4. **RSI (Black, Period=14)**: The Relative Strength Index indicates potential market reversals when the market is overbought or oversold.\n5. **MACD (Fast=12, Slow=26, Signal=9)**: The Moving Average Convergence Divergence shows changes in the strength, direction, momentum, and duration of a trend.\n\n#### Current Chart Observa

In [117]:
# эта ячейка берет ответ из предыдущей ячейки и вставляет его в маркдаун ячейку ниже, чтобы было удобно читать.

# Запустите следующий код в Jupyter Notebook, чтобы создать функцию, которая будет добавлять новую ячейку с содержимым:
from IPython.display import display, Markdown, Javascript

def create_markdown_cell(content):
    display(Markdown(content))

# Создайте функцию, которая удаляет следующую ячейку, если она имеет тип Markdown:
def delete_next_markdown_cell():
    display(Javascript("""
    var currentCell = Jupyter.notebook.get_selected_index();
    var nextCell = Jupyter.notebook.get_cell(currentCell + 1);
    if (nextCell && nextCell.cell_type === 'markdown') {
        Jupyter.notebook.delete_cell(currentCell + 1);
    }
    """))

# Извлечение значения поля content
content = j['choices'][0]['message']['content']

# Удаление следующей Markdown ячейки, если она существует
delete_next_markdown_cell()

# Создание новой markdown ячейки с содержимым
create_markdown_cell(content)

<IPython.core.display.Javascript object>

### Analysis of the Minute Candlestick Chart

#### Background and Indicators
1. **Bollinger Bands (Black, Period=20)**: These provide a measure of volatility, indicating potential overbought and oversold conditions.
2. **VWAP (Blue)**: The Volume Weighted Average Price helps understand the true average price a stock has traded at throughout the day, considering volume.
3. **ATR (Red, Period=10)**: The Average True Range measures market volatility.
4. **RSI (Black, Period=14)**: The Relative Strength Index indicates potential market reversals when the market is overbought or oversold.
5. **MACD (Fast=12, Slow=26, Signal=9)**: The Moving Average Convergence Divergence shows changes in the strength, direction, momentum, and duration of a trend.

#### Current Chart Observations
- **Candlestick Movement**: A recent downtrend is observed, with prices currently near the lower Bollinger Band, indicating a potential oversold condition.
- **VWAP**: The price is trading below the VWAP line, signifying a bearish market trend.
- **ATR**: The ATR is relatively high, indicating increased volatility.
- **Volumes**: The volume shows a spike corresponding to the recent downtrend, which suggests strong selling pressure.
- **RSI**: The RSI is below 30, indicating that the asset may be oversold.
- **MACD**: The MACD line is below the signal line, which suggests a bearish signal, although the histogram shows signs of narrowing, indicating potential weakening of the downtrend.

### Forecast for Next 5 Minutes
- **Price Movements**: Given that the price is near the lower Bollinger Band and the RSI indicates oversold conditions, there is a probability of a short-term reversal or stabilization.
- **Volumes**: Given the high volume, it may be indicative of the last phases of the selling pressure before a potential reversal.
- **ATR**: High ATR suggests we should prepare for continued volatility, though not necessarily in a single direction.
- **MACD**: Bearish, but potentially indicating a slowing down in bearish momentum.

#### Estimated Closing Price Range and Probabilities
- **Predicted Closing Price Range**: Given the high volatility, we can estimate the closing price to be within approximately 67600 to 67850.
- **Probabilities**:
  - **Probability of Increase**: 60% - An increase is possible due to the oversold RSI and proximity to the lower Bollinger Band, indicating a potential short-term reversal.
  - **Probability of Decrease**: 40% - There is still a risk of further decline, given the prevailing bearish trend in the MACD, but this is less likely due to the possible exhaustion of selling pressure.

### Conclusion
While the recent trend has been downward, current indicators suggest the possibility of short-term stabilization or a minor reversal. The market is currently highly volatile, so caution should be exercised. Always ensure risk management strategies are in place when trading in volatile conditions.

In [118]:
content

'### Analysis of the Minute Candlestick Chart\n\n#### Background and Indicators\n1. **Bollinger Bands (Black, Period=20)**: These provide a measure of volatility, indicating potential overbought and oversold conditions.\n2. **VWAP (Blue)**: The Volume Weighted Average Price helps understand the true average price a stock has traded at throughout the day, considering volume.\n3. **ATR (Red, Period=10)**: The Average True Range measures market volatility.\n4. **RSI (Black, Period=14)**: The Relative Strength Index indicates potential market reversals when the market is overbought or oversold.\n5. **MACD (Fast=12, Slow=26, Signal=9)**: The Moving Average Convergence Divergence shows changes in the strength, direction, momentum, and duration of a trend.\n\n#### Current Chart Observations\n- **Candlestick Movement**: A recent downtrend is observed, with prices currently near the lower Bollinger Band, indicating a potential oversold condition.\n- **VWAP**: The price is trading below the VWAP

In [119]:
# в этой ячейке делаем второй запрос к API, в котором просим предыдущий ответ оформить в виде json.

payload = {
  "model": "gpt-3.5-turbo",# берем модель 3.5 турбо, т.к. она в 10 раз дешевле gpt-4o и с задачей справляется.
  "response_format": { "type": "json_object" },
  "messages": [
    {
      "role": "system", 
      "content": """You are an LLM assistant specialized in converting financial analysis text into structured JSON format.
 Your task is to extract key numerical values and probabilities from the provided analysis and format
 them into a JSON object with the following fields: 'h' for the predicted highest price, 'l' for the
 predicted lowest price, 'p_up' for the probability of price increase, 'p_down' for the probability
 of price decrease, and 'error' for any potential issues or discrepancies noted during extraction.
 Ensure the JSON output accurately reflects the information in the analysis, adheres to the following
 JSON Schema:

```
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "h": {
      "type": "number",
      "description": "Predicted highest price"
    },
    "l": {
      "type": "number",
      "description": "Predicted lowest price"
    },
    "p_up": {
      "type": "number",
      "description": "Probability of price increase",
      "minimum": 0,
      "maximum": 100
    },
    "p_down": {
      "type": "number",
      "description": "Probability of price decrease",
      "minimum": 0,
      "maximum": 100
    },
    "error": {
      "type": "string",
      "description": "Error message if any issue is found"
    }
  },
  "additionalProperties": false
}
```

Respond only in JSON format without any additional text, as your response will be processed automatically by a program."""
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": content
        }
      ],
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

{'id': 'chatcmpl-9VSW5rH5kCbO3Kzr8NwZxyXnih0gU',
 'object': 'chat.completion',
 'created': 1717284813,
 'model': 'gpt-3.5-turbo-0125',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{\n    "h": 67850,\n    "l": 67600,\n    "p_up": 60,\n    "p_down": 40,\n    "error": ""\n}'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 987,
  'completion_tokens': 39,
  'total_tokens': 1026},
 'system_fingerprint': None}

In [120]:
json_string = j['choices'][0]['message']['content']

# Преобразование строки JSON в объект Python (словарь)
data = json.loads(json_string)

# Чтение значения 'h'
data["h"], data['l'], data['p_up'], data['p_down'], data['error']

(67850, 67600, 60, 40, '')